# Introduction

In this notebook, I will attempt to tweak the multi-level perceptron model provided in the sample notebook, and report my findings. The list of tweaks are listed as follow (keeping everything else default):<br>

1) Fully connected layer output shape: (512, 256)(default) vs (256, 128) and (1024, 512)<br>
2) Dropout rate: 0.2(default) vs 0.1 vs 0.5<br>
3) 2 hiddenlayer vs 3 hiddenlayers: (512, 256)(default) vs [1024, 512, 256].<br>
4) learning rate: 0.1(default) vs 0.3 and 0.1/epochs(time-based decay)

## Load Python libraries

Let's load some useful Python libraries:
1. numpy: structural data types
2. pandas: data loading and manipulation
3. matplotlib, sns: data visualization
4. sklearn: data analytics algorithms
5. Tensorflow: Deep Learning library
6. Keras: Wrapper for Tensorflow

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline  

import numpy as np

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers import Activation
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras import backend as K

from keras.datasets import cifar10


Using TensorFlow backend.


## Load data, perform reshape, normalization and one-hot encoding of labels

In [2]:
#number of final classes (in cifar10, there are 10 classes)
nb_classes = 10
nb_epoch = 100
batch_size = 128

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X_train = X_train.reshape(50000, 32 * 32 * 3)
X_test = X_test.reshape(10000, 32 * 32 * 3)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.0
X_test /= 255.0

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

## Define plot function:

In [3]:
def plot_train_acc(i, historyList):
    fig = plt.figure()
    for index, his in enumerate(historyList):
        plt.plot(range(nb_epoch),his.history['acc'],label='training'+str(index))
    plt.legend(loc=0)
    plt.xlabel('epochs')
    plt.ylabel('training_accuracy')
    plt.xlim([1,nb_epoch])
    plt.grid(True)
    plt.title("Training Accuracy Comparison")
    plt.show()
    fig.savefig('img/'+str(i)+'-training-accuracy.png')
    plt.close(fig)
    
def plot_val_acc(i, historyList):
    fig = plt.figure()
    for index, his in enumerate(historyList):
        plt.plot(range(nb_epoch),his.history['val_acc'],label='validation'+str(index))
    plt.legend(loc=0)
    plt.xlabel('epochs')
    plt.ylabel('validation_accuracy')
    plt.xlim([1,nb_epoch])
    plt.grid(True)
    plt.title("Validation Accuracy Comparison")
    plt.show()
    fig.savefig('img/'+str(i)+'-validation-accuracy.png')
    plt.close(fig)
    
def saveHistory(history, filename):
    import json
    json.dump(history.history, open('json_history/'+filename+'.json', 'w+'))

## 1) Different Output Shape:

Output shape refers to the number of neurons in each layer. In the sample, there are 4 layers:
Input Layer, 2 hidden layers with output shape: 512 and 256 respectively, and an Output Layer with outputshape: 10

### Define base model function:

In [4]:
def MLP_shape(shape1, shape2):
    model = Sequential()
    model.add(Dense(shape1, input_shape=X_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(shape2))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='sgd',
                  metrics=['accuracy'])
    #model.summary()
    return(model)


#### Create Default Model (based on the sample notebook)

In [5]:
modelDef = MLP_shape(512, 256) #default model
historyDef = modelDef.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(X_test, Y_test))
jsaveHistory(historyDef,'historyDef')

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 18s 359us/step - loss: 2.0508 - acc: 0.2534 - val_loss: 1.8805 - val_acc: 0.3403
Epoch 2/100
50000/50000 [==============================] - 17s 332us/step - loss: 1.8722 - acc: 0.3338 - val_loss: 1.7928 - val_acc: 0.3709
Epoch 3/100
50000/50000 [==============================] - 17s 333us/step - loss: 1.8027 - acc: 0.3619 - val_loss: 1.7317 - val_acc: 0.3901
Epoch 4/100
50000/50000 [==============================] - 18s 364us/step - loss: 1.7557 - acc: 0.3799 - val_loss: 1.6864 - val_acc: 0.4079
Epoch 5/100
50000/50000 [==============================] - 17s 346us/step - loss: 1.7184 - acc: 0.3927 - val_loss: 1.6523 - val_acc: 0.4235
Epoch 6/100
50000/50000 [==============================] - 18s 350us/step - loss: 1.6857 - acc: 0.4048 - val_loss: 1.6243 - val_acc: 0.4346
Epoch 7/100
50000/50000 [==============================] - 18s 350us/step - loss: 1.6584 - acc: 0.4139 - val_l

50000/50000 [==============================] - 17s 349us/step - loss: 1.1939 - acc: 0.5797 - val_loss: 1.3245 - val_acc: 0.5344
Epoch 59/100
50000/50000 [==============================] - 17s 330us/step - loss: 1.1915 - acc: 0.5786 - val_loss: 1.2967 - val_acc: 0.5401
Epoch 60/100
50000/50000 [==============================] - 17s 334us/step - loss: 1.1855 - acc: 0.5827 - val_loss: 1.3083 - val_acc: 0.5447
Epoch 61/100
50000/50000 [==============================] - 17s 330us/step - loss: 1.1814 - acc: 0.5836 - val_loss: 1.2905 - val_acc: 0.5425
Epoch 62/100
50000/50000 [==============================] - 16s 328us/step - loss: 1.1756 - acc: 0.5853 - val_loss: 1.3295 - val_acc: 0.5249
Epoch 63/100
50000/50000 [==============================] - 16s 327us/step - loss: 1.1706 - acc: 0.5840 - val_loss: 1.2873 - val_acc: 0.5420
Epoch 64/100
50000/50000 [==============================] - 16s 329us/step - loss: 1.1669 - acc: 0.5886 - val_loss: 1.3309 - val_acc: 0.5191
Epoch 65/100
50000/50000 [

NameError: name 'json' is not defined

### Generate 2 other models with different output shapes and plot the graphs:

In [ ]:
model1 = MLP_shape(256, 128)
model2 = MLP_shape(1024, 512)

history1 = model1.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(X_test, Y_test))
saveHistory(history1,'history1')

history2 = model2.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(X_test, Y_test))
saveHistory(history2,'history2')

plot_train_acc(1, [historyDef, history1, history2])
plot_val_acc(2, [historyDef, history1, history2])


## 2) Different dropout rate:

### Define base model function:

In [ ]:
def MLP_drop_rate(drop_rate):
    model = Sequential()
    model.add(Dense(512, input_shape=X_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Dropout(drop_rate))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(drop_rate))
    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='sgd',
                  metrics=['accuracy'])
    #model.summary()
    return(model)

### Generate 3 different models with different drop rate and plot the graphs:

In [ ]:
model3 = MLP_drop_rate(0.1)
model4 = MLP_drop_rate(0.5)

history3 = model3.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(X_test, Y_test))
saveHistory(history3,'history3')
history4 = model4.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(X_test, Y_test))
saveHistory(history4,'history4')

plot_train_acc(3, [historyDef, history3, history4])
plot_val_acc(4, [historyDef, history3, history4])

## 3) Different number of layers:

In [ ]:
# model 5 has 3 hiden layers
model5 = Sequential()
model5.add(Dense(1024, input_shape=X_train.shape[1:]))
model5.add(Activation('relu'))
model5.add(Dropout(0.2))
model5.add(Dense(512))
model5.add(Activation('relu'))
model5.add(Dropout(0.2)
model5.add(Dense(256))
model5.add(Activation('relu'))
model5.add(Dropout(0.2))
model5.add(Dense(10))
model5.add(Activation('softmax'))

model5.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

history5 = model5.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(X_test, Y_test))
saveHistory(history5,'history5')
           
plot_train_acc(5 [historyDef, history5])
plot_val_acc(6, [historyDef, history5])

## 4) Different Learning Rate

### Define base model function:

In [ ]:
def MLP_learningRate(lr, decay):
    model = Sequential()
    model.add(Dense(512, input_shape=X_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10))
    model.add(Activation('softmax'))
    
    sgd = SGD(lr=lr, momentum=0.0, decay=decay, nesterov=False)
    model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    #model.summary()
    return(model)

In [ ]:
model6 = MLP_learningRate(0.3, 0.0)
model7 = MLP_learningRate(0.1, 0.1/nb_epoch)

history6 = model6.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(X_test, Y_test))
saveHistory(history6,'history6')
history7 = model7.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(X_test, Y_test))
saveHistory(history7,'history7')

plot_train_acc(7, [historyDef, history6, history7])
plot_val_acc(8, [historyDef, history6, history7])

## 5) Different Activation Functions:

### Define base model function:

In [ ]:
def MLP_act_func(act_func):
    model = Sequential()
    model.add(Dense(512, input_shape=X_train.shape[1:]))
    model.add(Activation(act_func))
    model.add(Dropout(0.2))
    model.add(Dense(256))
    model.add(Activation(act_func))
    model.add(Dropout(0.2))
    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='sgd',
                  metrics=['accuracy'])
    #model.summary()
    return(model)

In [ ]:
model8 = MLP_act_func('sigmoid')
model9 = MLP_act_func('prelu')
model10 = MLP_act_func('softplus')

history8 = model8.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(X_test, Y_test))
saveHistory(history8,'history8')
history9 = model9.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(X_test, Y_test))
saveHistory(history9,'history9')
history10 = model10.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(X_test, Y_test))
saveHistory(history10,'history10')

plot_train_acc(7, [historyDef, history8, history9, history10])
plot_val_acc(8, [historyDef, history8, history9, history10])

## 6) Different Loss Function:
Since there are 10 classes in this dataset, 'categorical cross entropy' is the only option

In [ ]:
def MLP_loss_func(loss_func):
    model = Sequential()
    model.add(Dense(512, input_shape=X_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(loss=loss_func,
                  optimizer='sgd',
                  metrics=['accuracy'])
    #model.summary()
    return(model)

In [ ]:
model11 = MLP_loss_func('sparse_categorical_crossentropy')


history11 = model11.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(X_test, Y_test))
saveHistory(history11,'history11')

plot_train_acc(10, [historyDef, history11])
plot_val_acc(12, [historyDef, history11])

You can add your analysis and results using these markdown cells.
Some of the things you can play with for your models include
1. Effects of varying number of hidden layers
2. Effects of varying number of neurons in the hidden layers
3. Effects of different learning rates
4. Effects of other parameters (training method, activation function, loss function etc)

You can also change the metric for classifier performance by using other measures such as mean squared error, confusion matrices etc.

Here are some useful links for learning more about creating NN architectures:
1. Keras - http://keras.io
2. Tensorflow - http://www.tensorflow.org/
3. Python numpy tutorial -  http://cs231n.github.io/python-numpy-tutorial/
